<div style="color:rgb(0,0,255);font-size: 40px;font-weight:700;">
MAIN SETTINGS
</div>

In [ ]:
###SETTINGS###

import os
import re
import time
import subprocess
import shutil
from concurrent.futures import ThreadPoolExecutor, as_completed
from getpass import getpass
from urllib.parse import urlencode

# Platform detection
ON_KAGGLE = os.path.exists('/kaggle')
ON_COLAB = 'COLAB_RELEASE_TAG' in os.environ or os.path.exists('/content')
ON_VAST = any(k in os.environ for k in ("VAST_CONTAINERLABEL", "VAST_TCP_PORT_22", "CONTAINER_ID")) or os.path.exists('/workspace')


MAX_PARALLEL_DOWNLOADS = max(1, int(os.environ.get("MAX_PARALLEL_DOWNLOADS", "3")))
MIN_VALID_FILE_BYTES = int(os.environ.get("MIN_VALID_FILE_BYTES", "1000000"))

if shutil.which("aria2c") is None:
    print("aria2c not found → installing...")
    try:
        subprocess.run(["apt", "update", "-qq"], check=True, capture_output=True)
        result = subprocess.run(["apt", "install", "-y", "-qq", "aria2"], capture_output=True, text=True)
        if result.returncode == 0:
            print("aria2c installed successfully")
        else:
            print("Install failed (code {}):".format(result.returncode))
            print("stderr:", result.stderr.strip())
    except subprocess.CalledProcessError as e:
        print(f"apt error (code {e.returncode}): {e.stderr}")
    except Exception as e:
        print(f"Unexpected error: {e}")
else:
    print("aria2c already available")

# Determining the working directory
possible_bases = [
    "/workspace",       # Vast.ai / RunPod
    "/kaggle/working",  # Kaggle
    "/content",         # Google Colab
]

BASE_DIR = None
for path in possible_bases:
    if os.path.isdir(path):
        BASE_DIR = path
        break

if BASE_DIR is None:
    BASE_DIR = os.getcwd()
    print("WARNING: Known directory not found:", BASE_DIR)

print("Working directory:", BASE_DIR)

# Configuration
FORGE_DIR        = os.path.join(BASE_DIR, "stable-diffusion-webui-forge")
MODELS_DIR       = os.path.join(BASE_DIR, "stable-diffusion-webui-forge", "models", "Stable-diffusion")
LORA_DIR         = os.path.join(BASE_DIR, "stable-diffusion-webui-forge", "models", "Lora")
CONTROLNET_DIR   = os.path.join(FORGE_DIR, "extensions", "sd-webui-controlnet")
CONTROLNET_MODELS_DIR = os.path.join(CONTROLNET_DIR, "models")
EXTENSIONS_DIR   = os.path.join(FORGE_DIR, "extensions")
OUTPUTS_DIR      = os.path.join(FORGE_DIR, "outputs")
VOLUME_DIR       = os.path.join(BASE_DIR, "volume")
GEN_DIR          = os.path.join(BASE_DIR, "gen")

for d in [MODELS_DIR, LORA_DIR, CONTROLNET_DIR, CONTROLNET_MODELS_DIR, EXTENSIONS_DIR, OUTPUTS_DIR, VOLUME_DIR, GEN_DIR]:
    os.makedirs(d, exist_ok=True)

# Dependencies used by generation cell
for pkg in ["openpyxl", "requests"]:
    try:
        __import__(pkg)
    except Exception:
        print(f"Installing missing dependency: {pkg}")
        subprocess.run(["python", "-m", "pip", "install", "-q", pkg], check=False)


def get_secret(name: str):
    """Get secret from env/Kaggle/Colab only."""
    value = os.environ.get(name)
    if value:
        return value.strip(), "env"

    # Kaggle secrets
    if ON_KAGGLE:
        try:
            from kaggle_secrets import UserSecretsClient
            value = UserSecretsClient().get_secret(name)
            if value:
                return value.strip(), "kaggle_secrets"
        except Exception:
            pass

    # Colab secrets panel: from google.colab import userdata
    if ON_COLAB:
        try:
            from google.colab import userdata
            value = userdata.get(name)
            if value:
                return value.strip(), "colab_userdata"
        except Exception:
            pass

    return None, None


CIVITAI_TOKEN, CIVITAI_SRC = get_secret("CIVITAI_TOKEN")
HF_TOKEN, HF_SRC = get_secret("HF_TOKEN")

if not CIVITAI_TOKEN:
    manual_civitai = getpass("Enter CIVITAI_TOKEN (leave blank to skip): ").strip()
    if manual_civitai:
        CIVITAI_TOKEN, CIVITAI_SRC = manual_civitai, "manual_input"

if not HF_TOKEN:
    manual_hf = getpass("Enter HF_TOKEN (leave blank to skip): ").strip()
    if manual_hf:
        HF_TOKEN, HF_SRC = manual_hf, "manual_input"

TOKENS = {}
if CIVITAI_TOKEN:
    TOKENS["CIVITAI"] = CIVITAI_TOKEN
if HF_TOKEN:
    TOKENS["HF_TOKEN"] = HF_TOKEN

print("Token sources:")
print(f"  CIVITAI_TOKEN: {CIVITAI_SRC or 'not found'}")
print(f"  HF_TOKEN: {HF_SRC or 'not found'}")
if ON_VAST:
    print("Vast.ai tip: add CIVITAI_TOKEN/HF_TOKEN in template env vars, restart container, then rerun this cell.")

if not CIVITAI_TOKEN:
    print("CivitAI token not found")
if not HF_TOKEN:
    print("HF token not found")
if not TOKENS:
    raise RuntimeError("No tokens were provided. Set secrets or enter at least one token (CivitAI or HF).")


def _prepare_download_url(url, token):
    """CivitAI download works more reliably with token as query param."""
    if token and "civitai.com/api/download/models" in url and "token=" not in url:
        sep = "&" if "?" in url else "?"
        return f"{url}{sep}{urlencode({'token': token})}"
    return url


def _looks_valid_file(path, min_bytes=MIN_VALID_FILE_BYTES):
    return os.path.exists(path) and os.path.getsize(path) > min_bytes


def _human_mb(num_bytes):
    return f"{num_bytes / (1024 * 1024):.1f} MB"


def _estimate_expected_mb(label):
    # Examples: "151 MB", "6,46 GB"
    match = re.search(r"(\d+[\.,]?\d*)\s*(MB|GB)", label, re.IGNORECASE)
    if not match:
        return None
    value = float(match.group(1).replace(',', '.'))
    unit = match.group(2).upper()
    return value * (1024 if unit == "GB" else 1)


def _size_sanity_warning(path, expected_mb, tolerance=0.7):
    if expected_mb is None or not os.path.exists(path):
        return
    actual_mb = os.path.getsize(path) / (1024 * 1024)
    if actual_mb < expected_mb * tolerance:
        print(f"  WARNING: file size looks low ({actual_mb:.1f} MB vs expected ~{expected_mb:.1f} MB)")


def _has_min_free_disk(path, required_mb, reserve_mb=1024):
    if required_mb is None:
        return True
    usage = shutil.disk_usage(path)
    free_mb = usage.free / (1024 * 1024)
    return free_mb >= (required_mb + reserve_mb)


def _download_one(job, target_dir):
    label, url, filename, token_name = job
    token = TOKENS.get(token_name)
    output_path = os.path.join(target_dir, filename)

    expected_mb = _estimate_expected_mb(label)

    if _looks_valid_file(output_path):
        size = os.path.getsize(output_path)
        print(f"[SKIP] {label}: already exists ({_human_mb(size)})")
        _size_sanity_warning(output_path, expected_mb)
        return (label, True, "exists")

    if expected_mb is not None and not _has_min_free_disk(target_dir, expected_mb):
        return (label, False, "insufficient_disk")

    tmp_path = output_path + ".part"
    if os.path.exists(tmp_path):
        os.remove(tmp_path)

    final_url = _prepare_download_url(url, token if token_name == "CIVITAI" else None)

    cmd = [
        "aria2c",
        "--allow-overwrite=true",
        "--auto-file-renaming=false",
        "--continue=true",
        "--max-connection-per-server=16",
        "--split=16",
        "--min-split-size=1M",
        "--console-log-level=warn",
        "--summary-interval=1",
        "--check-certificate=false",
        "--out", os.path.basename(tmp_path),
        "--dir", target_dir,
        final_url,
    ]

    if token_name == "HF_TOKEN" and token:
        cmd.insert(-1, f"--header=Authorization: Bearer {token}")

    print(f"[DOWNLOADING] {label}")
    result = subprocess.run(cmd, text=True, capture_output=True)
    if result.returncode != 0:
        stderr = (result.stderr or "").strip()
        stdout = (result.stdout or "").strip()
        msg = stderr or stdout or f"aria2c exited {result.returncode}"
        if os.path.exists(tmp_path):
            os.remove(tmp_path)
        return (label, False, msg)

    if not _looks_valid_file(tmp_path):
        size = os.path.getsize(tmp_path) if os.path.exists(tmp_path) else 0
        if os.path.exists(tmp_path):
            os.remove(tmp_path)
        return (label, False, f"downloaded file too small ({_human_mb(size)})")

    os.replace(tmp_path, output_path)
    _size_sanity_warning(output_path, expected_mb)
    return (label, True, _human_mb(os.path.getsize(output_path)))


def run_download_list(download_list, target_dir, title):
    print(f"\n=== {title} ===")
    os.makedirs(target_dir, exist_ok=True)

    if not download_list:
        print("No items.")
        return

    workers = min(MAX_PARALLEL_DOWNLOADS, len(download_list))
    print(f"Parallel downloads: {workers}")

    ok = 0
    fail = 0

    with ThreadPoolExecutor(max_workers=workers) as ex:
        futures = [ex.submit(_download_one, job, target_dir) for job in download_list]
        for fut in as_completed(futures):
            label, success, info = fut.result()
            if success:
                ok += 1
                print(f"[OK]   {label} -> {info}")
            else:
                fail += 1
                print(f"[FAIL] {label} -> {info}")

    print(f"Done: OK={ok}, FAIL={fail}")
    if fail > 0:
        raise RuntimeError(f"Some downloads failed in {title}: {fail} item(s)")




<div style="color:rgb(0,0,255);font-size: 40px;font-weight:700;">
DOWNLOAD BLOCK
</div>

In [ ]:
### CONTROLNET INSTALL OPTIONS ###

# Option A (default): force clean reinstall
# Option B: set CONTROLNET_INSTALL_MODE = "update" for git pull in existing repo
# Option C: set CONTROLNET_INSTALL_MODE = "skip" to keep current state

CONTROLNET_REPO_URL = "https://github.com/Mikubill/sd-webui-controlnet"
CONTROLNET_INSTALL_MODE = os.environ.get("CONTROLNET_INSTALL_MODE", "reinstall").strip().lower()

if CONTROLNET_INSTALL_MODE not in {"reinstall", "update", "skip"}:
    raise ValueError("CONTROLNET_INSTALL_MODE must be one of: reinstall, update, skip")

print(f"ControlNet extension path: {CONTROLNET_DIR}")
print(f"Install mode: {CONTROLNET_INSTALL_MODE}")

if CONTROLNET_INSTALL_MODE == "skip":
    print("ControlNet install skipped")
else:
    if os.path.isdir(CONTROLNET_DIR) and CONTROLNET_INSTALL_MODE == "reinstall":
        print("Removing existing ControlNet directory...")
        shutil.rmtree(CONTROLNET_DIR)

    if not os.path.isdir(CONTROLNET_DIR):
        print("Cloning ControlNet repository...")
        subprocess.run(["git", "clone", CONTROLNET_REPO_URL, CONTROLNET_DIR], check=True)
    else:
        print("Updating ControlNet repository...")
        subprocess.run(["git", "-C", CONTROLNET_DIR, "pull", "--ff-only"], check=True)

os.makedirs(CONTROLNET_MODELS_DIR, exist_ok=True)
print("ControlNet repository is ready")
print(f"ControlNet models directory: {CONTROLNET_MODELS_DIR}")

#ControlNET models download

controlnet_models_to_download = [
    ("t2i-adapter_xl_openpose 151 MB", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/t2i-adapter_xl_openpose.safetensors", "t2i-adapter_xl_openpose.safetensors", "HF_TOKEN"),
    ("t2i-adapter_xl_canny 148 MB", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/t2i-adapter_xl_canny.safetensors", "t2i-adapter_xl_canny.safetensors", "HF_TOKEN"),
    ("t2i-adapter_xl_sketch 148 MB", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/t2i-adapter_xl_sketch.safetensors", "t2i-adapter_xl_sketch.safetensors", "HF_TOKEN"),
    ("t2i-adapter_diffusers_xl_depth_midas 151 MB", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/t2i-adapter_diffusers_xl_depth_midas.safetensors", "t2i-adapter_diffusers_xl_depth_midas.safetensors", "HF_TOKEN"),
    ("t2i-adapter_diffusers_xl_depth_zoe 151 MB", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/t2i-adapter_diffusers_xl_depth_zoe.safetensors", "t2i-adapter_diffusers_xl_depth_zoe.safetensors", "HF_TOKEN"),
    ("t2i-adapter_diffusers_xl_lineart 151 MB", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/t2i-adapter_diffusers_xl_lineart.safetensors", "t2i-adapter_diffusers_xl_lineart.safetensors", "HF_TOKEN"),
]

run_download_list(controlnet_models_to_download, CONTROLNET_MODELS_DIR, "ControlNet")

#Checkpoints download

models_to_download = [
    ("WAI ILL V16.0 6,46 GB", "https://civitai.com/api/download/models/2514310?type=Model&format=SafeTensor&size=pruned&fp=fp16", "wai_v160.safetensors", "CIVITAI"),
]

run_download_list(models_to_download, MODELS_DIR, "Checkpoints")

#LoRa download

lora_to_download = [
    ("Detailer IL V2 218 MB",        "https://civitai.com/api/download/models/1736373?type=Model&format=SafeTensor",    "detailer_v2_il.safetensors",     "CIVITAI"),
    ("Realistic filter V1 55 MB",    "https://civitai.com/api/download/models/1124771?type=Model&format=SafeTensor",    "realistic_filter_v1_il.safetensors", "CIVITAI"),
    ("Hyperrealistic V4 ILL 435 MB", "https://civitai.com/api/download/models/1914557?type=Model&format=SafeTensor",    "hyperrealistic_v4_ill.safetensors",  "CIVITAI"),
    ("Niji semi realism V3.5 ILL 435 MB", "https://civitai.com/api/download/models/1882710?type=Model&format=SafeTensor", "niji_v35.safetensors", "CIVITAI"),
    ("ATNR Style ILL V1.1 350 MB", "https://civitai.com/api/download/models/1711464?type=Model&format=SafeTensor", "atnr_style_ill_v1.1.safetensors", "CIVITAI"),
    ("Face Enhancer Ill 218 MB", "https://civitai.com/api/download/models/1839268?type=Model&format=SafeTensor", "face_enhancer_ill.safetensors", "CIVITAI"),
    ("Smooth Detailer Booster V4 243 MB", "https://civitai.com/api/download/models/2196453?type=Model&format=SafeTensor", "smooth_detailer_booster_v4.safetensors", "CIVITAI"),
    ("USNR Style V-pred 157 MB", "https://civitai.com/api/download/models/2555444?type=Model&format=SafeTensor", "usnr_style.safetensors", "CIVITAI"),
    ("748cm Style V1 243 MB", "https://civitai.com/api/download/models/1056404?type=Model&format=SafeTensor", "748cm_style_v1.safetensors", "CIVITAI"),
    ("Velvet's Mythic Fantasy Styles IL 218 MB", "https://civitai.com/api/download/models/2620790?type=Model&format=SafeTensor", "velvets_styles.safetensors", "CIVITAI"),
    ("Pixel Art Style IL V7 435 MB", "https://civitai.com/api/download/models/2661972?type=Model&format=SafeTensor", "pixel_art.safetensors", "CIVITAI"),
]

run_download_list(lora_to_download, LORA_DIR, "LoRA")



<div style="color:rgb(0,0,255);font-size: 40px;font-weight:700;">
ARCHIVE+OUTPUT
</div>

In [ ]:
### API GENERATION FROM XLSX ###

import os
import re
import json
import glob
import time
import shutil
import zipfile
import base64
from datetime import datetime

import requests
from openpyxl import load_workbook

API_BASE = os.environ.get("FORGE_API_BASE", "http://127.0.0.1:17860")
TXT2IMG_URL = f"{API_BASE}/sdapi/v1/txt2img"
OPENAPI_URL = f"{API_BASE}/openapi.json"

PROMPTS_CANDIDATES = [
    os.path.join(GEN_DIR, "prompts.xlsx"),
    os.path.join(GEN_DIR, "prompts.xlxs"),
    os.path.join(GEN_DIR, "Prompts.xlsx"),
    os.path.join(GEN_DIR, "Prompts.xlxs"),
    os.path.join(BASE_DIR, "prompts.xlsx"),
    os.path.join(BASE_DIR, "prompts.xlxs"),
    os.path.join(BASE_DIR, "Prompts.xlsx"),
    os.path.join(BASE_DIR, "Prompts.xlxs"),
    "/workspace/gen/prompts.xlsx",
    "/workspace/gen/prompts.xlxs",
]

if ON_COLAB:
    # Colab manual upload often lands in /content
    PROMPTS_CANDIDATES.extend([
        "/content/prompts.xlsx",
        "/content/prompts.xlxs",
        "/content/Prompts.xlsx",
        "/content/Prompts.xlxs",
    ])

if ON_KAGGLE:
    # Fixed Kaggle dataset path used in your workflow
    PROMPTS_CANDIDATES.append("/kaggle/input/datasets/sokolenkotimofei/prompts/Prompts.xlsx")

    # Additional Kaggle dataset input path patterns (fallback)
    for pattern in [
        "/kaggle/input/*/*/prompts/prompts.xlsx",
        "/kaggle/input/*/*/prompts/prompts.xlxs",
        "/kaggle/input/*/*/prompts/Prompts.xlsx",
        "/kaggle/input/*/*/prompts/Prompts.xlxs",
        "/kaggle/input/*/prompts/prompts.xlsx",
        "/kaggle/input/*/prompts/prompts.xlxs",
        "/kaggle/input/*/prompts/Prompts.xlsx",
        "/kaggle/input/*/prompts/Prompts.xlxs",
    ]:
        PROMPTS_CANDIDATES.extend(sorted(glob.glob(pattern)))

# Keep order, remove duplicates
PROMPTS_CANDIDATES = list(dict.fromkeys(PROMPTS_CANDIDATES))

API_READY_TIMEOUT = int(os.environ.get("FORGE_API_READY_TIMEOUT", "180"))
API_READY_INTERVAL = float(os.environ.get("FORGE_API_READY_INTERVAL", "3"))
FORGE_BG_LOG = os.path.join(VOLUME_DIR, "forge_background.log")


def diagnose_api_unreachable(base_url: str):
    print("\n[DIAG] Forge API still unavailable.")
    print("[DIAG] Expected run order for Kaggle/Colab: 1) SETTINGS → 2) DOWNLOAD BLOCK → 3) OPTIONAL BOOTSTRAP (cell 7) → 4) OPTIONAL START FORGE (cell 8) → 5) API GENERATION (this cell).")

    launch_script = os.path.join(FORGE_DIR, "webui.sh")
    if not os.path.exists(launch_script):
        print(f"[DIAG] webui.sh not found: {launch_script}")
        print("[DIAG] Run OPTIONAL BOOTSTRAP cell first (cell 7).")

    if os.path.exists(FORGE_BG_LOG):
        print(f"[DIAG] Found Forge background log: {FORGE_BG_LOG}")
        try:
            with open(FORGE_BG_LOG, "r", encoding="utf-8", errors="ignore") as f:
                tail_lines = f.readlines()[-20:]
            print("[DIAG] Last Forge log lines:")
            print("".join(tail_lines).strip() or "[DIAG] (log is empty)")
        except Exception as log_err:
            print(f"[DIAG] Failed to read Forge log: {log_err}")
    else:
        print(f"[DIAG] Forge log not found: {FORGE_BG_LOG}")
        print("[DIAG] Start OPTIONAL START FORGE IN BACKGROUND cell (cell 8), then retry this cell.")

def wait_for_api_ready(base_url: str, timeout_s: int = API_READY_TIMEOUT, interval_s: float = API_READY_INTERVAL):
    start = time.time()
    last_error = None
    while (time.time() - start) < timeout_s:
        try:
            r = requests.get(f"{base_url}/sdapi/v1/progress", timeout=10)
            if r.status_code < 500:
                return True
            last_error = f"HTTP {r.status_code}"
        except Exception as e:
            last_error = str(e)
        time.sleep(interval_s)
    diagnose_api_unreachable(base_url)
    raise RuntimeError(
        f"Forge API is not reachable at {base_url} after {timeout_s}s. "
        f"Last error: {last_error}. "
        "Run cells 7 and 8 first on Kaggle/Colab, then rerun this API cell."
    )


LOG_PATH = os.path.join(VOLUME_DIR, "log.txt")
API_IMAGES_DIR = os.path.join(OUTPUTS_DIR, "api_generated")
os.makedirs(API_IMAGES_DIR, exist_ok=True)


CONFLICT_RULES = [
    (("hr_resize_x", "hr_resize_y"), ("hr_scale",)),
]


def log_line(text: str):
    ts = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    line = f"[{ts}] {text}"
    print(line)
    with open(LOG_PATH, "a", encoding="utf-8") as f:
        f.write(line + "\n")


def normalize_value(value):
    if isinstance(value, str):
        stripped = value.strip()
        lowered = stripped.lower()
        if lowered in {"", "null", "none", "nan"}:
            return None
        if lowered in {"true", "yes", "1"}:
            return True
        if lowered in {"false", "no", "0"}:
            return False
        if re.fullmatch(r"-?\d+", stripped):
            return int(stripped)
        if re.fullmatch(r"-?\d+\.\d+", stripped):
            return float(stripped)
        return stripped
    return value


def first_existing_path(candidates):
    for p in candidates:
        if os.path.exists(p):
            return p
    return None


def parse_workbook(path):
    wb = load_workbook(path, data_only=True)
    ws = wb.active

    instruction_parts = [str(v).strip() for v in ws[1] if v is not None and str(v).strip()]
    if not instruction_parts:
        raise ValueError("Первая строка (инструкция) пустая")
    instruction = " ".join(instruction_parts)

    headers = [str(v).strip() if v is not None else "" for v in ws[2]]
    if not any(headers):
        raise ValueError("Вторая строка должна содержать имена переменных (заголовки столбцов)")

    rows = []
    for row_idx in range(3, ws.max_row + 1):
        row_values = [normalize_value(v) for v in next(ws.iter_rows(min_row=row_idx, max_row=row_idx, values_only=True))]
        if all(v is None for v in row_values):
            continue

        row_map = {}
        for idx, key in enumerate(headers):
            if not key:
                continue
            value = row_values[idx] if idx < len(row_values) else None
            if value is None:
                continue
            row_map[key] = value
        rows.append(row_map)

    if not rows:
        raise ValueError("Не найдено строк данных (начиная с 3-й строки)")

    return instruction, rows


def render_instruction(template: str, variables: dict):
    def repl(match):
        key = match.group(1)
        return str(variables.get(key, match.group(0)))
    return re.sub(r"\{([a-zA-Z0-9_]+)\}", repl, template)


def apply_conflict_rules(payload: dict):
    cleaned = dict(payload)
    for primary_keys, conflicting_keys in CONFLICT_RULES:
        primary_present = all((k in cleaned and cleaned[k] is not None) for k in primary_keys)
        if primary_present:
            for ck in conflicting_keys:
                cleaned.pop(ck, None)
    return cleaned


def fetch_txt2img_params_dump():
    try:
        response = requests.get(OPENAPI_URL, timeout=30)
        response.raise_for_status()
        spec = response.json()
        schemas = spec.get("components", {}).get("schemas", {})
        candidates = [
            "StableDiffusionTxt2ImgProcessingApi",
            "Txt2ImgRequest",
            "StableDiffusionProcessingTxt2Img",
        ]
        for name in candidates:
            if name in schemas:
                props = schemas[name].get("properties", {})
                return json.dumps({k: v.get("type", "unknown") for k, v in props.items()}, ensure_ascii=False, indent=2)
        return json.dumps(spec.get("paths", {}).get("/sdapi/v1/txt2img", {}), ensure_ascii=False, indent=2)
    except Exception as e:
        return f"Не удалось получить список переменных: {e}"


def save_images_from_response(images_b64, generation_idx):
    saved = 0
    for i, b64 in enumerate(images_b64, start=1):
        image_data = b64.split(",", 1)[-1]
        file_name = f"gen_{generation_idx:04d}_{i:02d}.png"
        file_path = os.path.join(API_IMAGES_DIR, file_name)
        with open(file_path, "wb") as f:
            f.write(base64.b64decode(image_data))
        saved += 1
    return saved


def archive_outputs(tag: str):
    stamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    archive_name = os.path.join(VOLUME_DIR, f"outputs_{tag}_{stamp}.zip")
    with zipfile.ZipFile(archive_name, "w", compression=zipfile.ZIP_DEFLATED) as zf:
        for root, _, files in os.walk(OUTPUTS_DIR):
            for file in files:
                full_path = os.path.join(root, file)
                rel_path = os.path.relpath(full_path, OUTPUTS_DIR)
                zf.write(full_path, rel_path)

    for entry in os.listdir(OUTPUTS_DIR):
        p = os.path.join(OUTPUTS_DIR, entry)
        if os.path.isdir(p):
            shutil.rmtree(p)
        else:
            os.remove(p)
    os.makedirs(API_IMAGES_DIR, exist_ok=True)
    log_line(f"ARCHIVE created: {archive_name}. OUTPUTS_DIR cleaned.")


wait_for_api_ready(API_BASE)

prompts_path = first_existing_path(PROMPTS_CANDIDATES)
if not prompts_path:
    raise FileNotFoundError(
        "Файл prompts.xlsx/prompts.xlxs не найден. Ожидались пути: " + ", ".join(PROMPTS_CANDIDATES)
    )

instruction_template, generation_rows = parse_workbook(prompts_path)
log_line(f"Loaded prompts file: {prompts_path}. Rows for generation: {len(generation_rows)}")

syntax_error_dumped = False
images_since_archive = 0

for generation_idx, row_values in enumerate(generation_rows, start=1):
    payload = dict(row_values)

    if "prompt" not in payload:
        payload["prompt"] = render_instruction(instruction_template, row_values)

    payload = {k: v for k, v in payload.items() if v is not None}
    payload = apply_conflict_rules(payload)

    try:
        response = requests.post(TXT2IMG_URL, json=payload, timeout=1800)
        if response.status_code >= 400:
            err_text = response.text[:1200]
            if not syntax_error_dumped:
                params_dump = fetch_txt2img_params_dump()
                log_line(f"#{generation_idx} FAIL syntax/validation: {response.status_code} {err_text}")
                log_line("AVAILABLE PARAMS DUMP START")
                with open(LOG_PATH, "a", encoding="utf-8") as f:
                    f.write(params_dump + "\n")
                log_line("AVAILABLE PARAMS DUMP END")
                syntax_error_dumped = True
            else:
                log_line(f"#{generation_idx} FAIL: {response.status_code} {err_text}")
            continue

        result = response.json()
        images = result.get("images", []) or []
        saved_now = save_images_from_response(images, generation_idx)
        images_since_archive += saved_now
        log_line(f"#{generation_idx} OK images={saved_now}")

        if images_since_archive >= 15:
            archive_outputs(tag=f"part_{generation_idx:04d}")
            images_since_archive = 0

    except requests.exceptions.Timeout:
        log_line(f"#{generation_idx} FAIL: timeout (possible heavy generation or API freeze)")
    except requests.exceptions.RequestException as e:
        reason = str(e)
        if "out of memory" in reason.lower() or "oom" in reason.lower():
            reason = "OOM"
        log_line(f"#{generation_idx} FAIL: {reason}")
    except Exception as e:
        log_line(f"#{generation_idx} FAIL: unexpected error: {e}")

if images_since_archive > 0:
    archive_outputs(tag="final")

log_line("Generation cycle completed")



In [ ]:
### OPTIONAL: KAGGLE/COLAB FORGE BOOTSTRAP ###

import os
import shutil
import subprocess

if not (ON_KAGGLE or ON_COLAB):
    print("Optional cell: предназначена только для Kaggle/Colab. Текущая платформа пропущена.")
else:
    required_packages = ["git", "python3-venv", "python3-pip"]
    print("Checking/installing platform dependencies...")
    subprocess.run(["apt", "update", "-qq"], check=False)
    subprocess.run(["apt", "install", "-y", "-qq", *required_packages], check=False)

    launch_script = os.path.join(FORGE_DIR, "webui.sh")
    git_head = os.path.join(FORGE_DIR, ".git", "HEAD")
    forge_ready = os.path.isfile(launch_script) and os.path.isfile(git_head)

    if forge_ready:
        print("WebUI Forge already exists and looks valid, skipping clone.")
    else:
        if os.path.isdir(FORGE_DIR):
            print("FORGE_DIR exists but WebUI Forge is incomplete/corrupted. Recreating...")
            shutil.rmtree(FORGE_DIR)

        print("Cloning WebUI Forge...")
        subprocess.run([
            "git", "clone", "https://github.com/lllyasviel/stable-diffusion-webui-forge", FORGE_DIR
        ], check=True)

        if not os.path.isfile(os.path.join(FORGE_DIR, "webui.sh")):
            raise FileNotFoundError("Clone completed but webui.sh not found. Check repository state.")

    print("Optional bootstrap finished.")



In [ ]:
### OPTIONAL: START WEBUI FORGE IN BACKGROUND ###

import os
import subprocess

FORGE_ARGS = "--xformers --api --cuda-malloc --cuda-stream --pin-shared-memory --theme dark --port 17860"

if not os.path.isdir(FORGE_DIR):
    raise FileNotFoundError(f"Forge directory not found: {FORGE_DIR}")

launch_script = os.path.join(FORGE_DIR, "webui.sh")
if not os.path.exists(launch_script):
    raise FileNotFoundError(f"Launch script not found: {launch_script}")

log_file = os.path.join(VOLUME_DIR, "forge_background.log")
cmd = f"cd {FORGE_DIR} && nohup bash webui.sh {FORGE_ARGS} > {log_file} 2>&1 &"

if ON_COLAB and os.geteuid() == 0:
    run_user = os.environ.get("FORGE_RUN_USER", "forge")

    user_exists = subprocess.run(["id", "-u", run_user], capture_output=True, text=True).returncode == 0
    if not user_exists:
        subprocess.run(["useradd", "-m", "-s", "/bin/bash", run_user], check=True)

    subprocess.run(["chown", "-R", f"{run_user}:{run_user}", FORGE_DIR], check=True)
    subprocess.run(["chown", "-R", f"{run_user}:{run_user}", VOLUME_DIR], check=True)

    subprocess.run(["su", "-s", "/bin/bash", "-c", cmd, run_user], check=True)
    print(f"WebUI Forge started in background mode as user '{run_user}'")
else:
    subprocess.run(["bash", "-lc", cmd], check=True)
    print("WebUI Forge started in background mode")

print(f"Log file: {log_file}")
print("Tip: use !tail -f", log_file)



In [ ]:
### OPTIONAL: WAIT FOR FORGE API CONNECTION ###

import time
import requests

API_BASE = os.environ.get("FORGE_API_BASE", "http://127.0.0.1:17860")
PING_URL = f"{API_BASE}/sdapi/v1/progress"
WAIT_TIMEOUT = int(os.environ.get("FORGE_WAIT_TIMEOUT", "1800"))
WAIT_INTERVAL = int(os.environ.get("FORGE_WAIT_INTERVAL", "30"))

print(f"Проверка связи с Forge API: {PING_URL}")
print(f"Интервал проверки: {WAIT_INTERVAL} сек, таймаут: {WAIT_TIMEOUT} сек")

start = time.time()
attempt = 0
last_error = None

while (time.time() - start) < WAIT_TIMEOUT:
    attempt += 1
    try:
        r = requests.get(PING_URL, timeout=10)
        if r.status_code < 500:
            elapsed = int(time.time() - start)
            print(f"✅ связь установлена: Forge API готов (попытка {attempt}, ~{elapsed} сек).")
            break
        last_error = f"HTTP {r.status_code}"
    except Exception as e:
        last_error = str(e)

    elapsed = int(time.time() - start)
    print(f"[{elapsed:>4} сек] API ещё не готов ({last_error}). Следующая проверка через {WAIT_INTERVAL} сек...")
    time.sleep(WAIT_INTERVAL)
else:
    raise RuntimeError(
        f"Не удалось дождаться Forge API за {WAIT_TIMEOUT} сек. Последняя ошибка: {last_error}"
    )
